In [3]:
import sys
print(sys.executable)


c:\Users\Minji Kang\AppData\Local\Programs\Python\Python311\python.exe


In [2]:
import networkx as nx
import matplotlib.pyplot as plt
from BNS_JT import cpm, variable, operation
from networkx.algorithms.flow import shortest_augmenting_path
from BNS_JT import brc
import numpy as np
import math
import random


ModuleNotFoundError: No module named 'BNS_JT'

In [ ]:
import matplotlib
print("Default Backend before BNS_JT import:", matplotlib.get_backend())
import BNS_JT
print("Backend after BNS_JT import:", matplotlib.get_backend())


Default Backend before BNS_JT import: TkAgg
Backend after BNS_JT import: TkAgg


In [ ]:
# 1. 그래프 정의
nodes = {
    "n1": (0, 0), "n2": (1.5, 5), "n3": (2.8, 2.5), "n4": (2.5, 0),
    "n5": (4.6, 7), "n6": (5.5, -0.5), "n7": (6.5, 3), "n8": (7, -1)
}

edges = {
    "e1": ("n1", "n2"), "e2": ("n1", "n3"), "e3": ("n1", "n4"),
    "e4": ("n2", "n3"), "e5": ("n3", "n4"), "e6": ("n5", "n2"),
    "e7": ("n5", "n3"), "e8": ("n3", "n7"), "e9": ("n3", "n6"), 
    "e10": ("n6", "n4"), "e11": ("n5", "n7"), "e12": ("n7", "n6"), 
    "e13": ("n6", "n8"), "e14": ("n7", "n8")
}

G = nx.Graph() 
for node, position in nodes.items():
    G.add_node(node, pos=position)
for edge_id, (u, v) in edges.items():
    pos_u, pos_v = nodes[u], nodes[v]
    distance = math.sqrt((pos_u[0] - pos_v[0])**2 + (pos_u[1] - pos_v[1])**2)
    G.add_edge(u, v, label=edge_id, flow=0, capacity=2, weight=distance)  # 노드와 노드 사이 거리를 가중치로 설정하여 최단경로탐색 시 더 짧은 거리를 우선순위로 함
                                                                          # Issue 1: edge 별 capacity 설정 기준이 없음. 임의의 capacity 2 설정
        
plt.figure(figsize=(5, 4))
pos = nx.get_node_attributes(G, 'pos')
edge_labels = nx.get_edge_attributes(G, 'label') 
nx.draw(G, pos, with_labels=True, node_size=800, node_color="lightblue", font_size=10, font_weight="bold")
nx.draw_networkx_edge_labels(G, pos, edge_labels=edge_labels, font_size=8, label_pos=0.5)
plt.title("Railway Network", fontsize=15)
plt.show()

#### Component events

The state of the five edges are represented by component events.<br>
The edges take binary-state, 0 for non-functional and 1 for functional.<br>

To use the BRC algorithm, component events need to be defined as a *variable.Variable* object.

In [ ]:
varis = {}
for k, v in edges.items():
    varis[k] = variable.Variable( name=k, values = [0, 1]) # values: edge flow capacity

print(varis['e1'])

probs = {'e1': {0: 0.01, 1: 0.99}, 'e2': {0: 0.01, 1: 0.99}, 'e3': {0: 0.05, 1: 0.95},
         'e4': {0: 0.05, 1: 0.95}, 'e5': {0: 0.10, 1: 0.90}, 'e6': {0: 0.01, 1: 0.99}
         'e7': {0: 0.01, 1: 0.99}, 'e8': {0: 0.01, 1: 0.99}, 'e9': {0: 0.05, 1: 0.95},
         'e10': {0: 0.01, 1: 0.99}, 'e11': {0: 0.01, 1: 0.99}, 'e12': {0: 0.05, 1: 0.95},
         'e13': {0: 0.01, 1: 0.99}, 'e14': {0: 0.01, 1: 0.99}}

SyntaxError: invalid syntax. Perhaps you forgot a comma? (299162190.py, line 8)

#### System event

System state is evaluated by network connectivity between a designed origin-destination (OD) pair. <br>
In other words, a system state is survival if the OD pair is connected; and failure otherwise.

<div style="margin-top:0;">
This leads to the system function below. <br>
<b>NB1</b> <b>The first input</b> of a system function must be <b>a dictionary of components state.</b><br> 
<b>NB2</b> <b>The outputs</b> must be three: 
<ul style="margin-top:0;">
<li> <em>system value:</em> any format (does not affect the BRC algorithm; just for information's sake); </li>
<li> <em>system state:</em> a string, either 's' or 'f'; and </li>
<li> <em>components state that guarantees an obtained system state:</em> a dictionary of affecting components OR (if such information unavailable) None.</li>
</ul>
</div>